## Random forest- 
Train multiple decision trees on subset of entire data, then aggregate the output and choose final output by majority voting.

- simple but effective algorithm

## implementing Ensemble model for classiffication with 3 different classsifiers.

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

svm_clf = SVC(probability=True)
logi_clf = LogisticRegression()
rand_clf = RandomForestClassifier()

voting_clf = VotingClassifier(
    estimators = [("lr",logi_clf),("rf",rand_clf),("svc",svm_clf)],
    voting="soft"
)

voting_clf

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [2]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X, y = make_moons(n_samples=200, noise=0.2, random_state=42)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)


In [3]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()),
                             ('svc', SVC(probability=True))],
                 voting='soft')

In [4]:
from sklearn.metrics import accuracy_score

for clf in (rand_clf, logi_clf, svm_clf,voting_clf):

    clf.fit(X_train,y_train)
    predi = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,predi))

    

RandomForestClassifier 1.0
LogisticRegression 0.85
SVC 0.95
VotingClassifier 0.925


In [5]:
import pandas as pd
df = pd.DataFrame(X)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       200 non-null    float64
 1   1       200 non-null    float64
dtypes: float64(2)
memory usage: 3.3 KB


In [6]:
df.describe()

,0,1
count,200.000000,200.000000
mean,0.508292,0.282361
std,0.901395,0.530963
min,-1.475982,-0.767866
25%,-0.120614,-0.158832
50%,0.527135,0.256909
75%,1.093087,0.693003
max,2.298011,1.486635


In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

## Out of Bag Evaluation- 

In bagging, some instances can be choosen multiple timea, while some can never be chosen, these instances that are not in training set are called Out Of Bag instances, 

So, if model havent seen these instances, we eveluate traned model on these instances which is called Out of Bag Evaluation.


In [8]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    oob_score=True
)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

print(bag_clf.oob_score_)

0.9375


## Random Forest-
Ensemble of trees trained using Bagging with usually max_samples=training set size.

In [9]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

rnd_clf

RandomForestClassifier(max_leaf_nodes=16, n_estimators=500, n_jobs=-1)

RandomForestClassifier = DecisionTreeClassifier + BaggingClassifier

### Above Random Forest classifier is internally same as following Bagging Classifier

In [10]:
bag_clf = BaggingClassifier(
DecisionTreeClassifier(splitter="random", max_leaf_nodes=16),
n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1)
bag_clf

BaggingClassifier(estimator=DecisionTreeClassifier(max_leaf_nodes=16,
                                                   splitter='random'),
                  n_estimators=500, n_jobs=-1)

## Feature importance (In random forest)

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09814692564451176
sepal width (cm) 0.022941396904589594
petal length (cm) 0.42033789658736703
petal width (cm) 0.45857378086353157
